In [25]:
from __future__ import print_function
import mysql.connector
from mysql.connector import errorcode
import requests
import re
from bs4 import BeautifulSoup
import random
import datetime

In [26]:
#создать базу
sql_lines = open("buses_create_script.sql").read() #отдельный скрипт в корне
cnx = mysql.connector.connect(user = "MNIXenus", password = "00mefomu", host = "127.0.0.1")
cursor = cnx.cursor()
try:
    lines = cursor.execute(sql_lines, multi = True) #попробовать выполнить
    for i in lines:
        print(i.fetchall())
except mysql.connector.Error as err:
    print(err.msg)
cursor.close()
cnx.close()

No result set to fetch from.


In [27]:
#шаблоны для ввода данных
add_route = ('INSERT INTO routes (idroutes, drivers) VALUES ("{}", "{}");')
add_stop = ('INSERT INTO stops (idstops, name) VALUES ("{}", "{}");')
add_drivers = ('INSERT INTO drivers(iddrivers, last_name, first_name, second_name) VALUES ("{}", "{}", "{}", "{}");')
add_sensor_data= ('INSERT INTO sensor_data (idSensor_data, route, stop, driver, time_arrival, time_departure) VALUES ("{}", "{}", "{}", "{}", "{}", "{}");')

In [29]:
#заполнить водителей
cnx = mysql.connector.connect(user = "MNIXenus", password = "00mefomu", host = "127.0.0.1", database = "buses")
cursor = cnx.cursor()
drivers = open('drivers.txt').read().split('\n')
drivers = [(i.split(' ')) for i in drivers]
for i in range(len(drivers)):
    driver = (i, drivers[i][0], drivers[i][1], drivers[i][2])
    cursor.execute(add_drivers.format(*driver))
cnx.commit()
cursor.close()
cnx.close()

In [10]:
#спарсить остановки
raw_html_stops = requests.get('https://www.bustime.ru/vladivostok/stop/')
findstops = re.compile('<div class="ui fluid vertical menu">(.+?)</div>', re.S)
stops = ''.join(findstops.findall(raw_html_stops.text))
stops = [BeautifulSoup(i, "html5lib").get_text() for i in stops.split('</a>')][:-1]

In [30]:
#Заполнить остановки
cnx = mysql.connector.connect(user = "MNIXenus", password = "00mefomu", host = "127.0.0.1", database = "buses")
cursor = cnx.cursor()
for i in range(len(stops)):
    stop = (i, stops[i])
    cursor.execute(add_stop.format(*stop))
cnx.commit()
cursor.close()
cnx.close()

In [12]:
#Спарсить маршруты
raw_html_routes = requests.get('https://www.bustime.ru/vladivostok/')
findroutes = re.compile('<div class="icon mode_0 ">(.+?)</a></div>', re.S)
getroutes = re.compile('href="#bus-(.+?)"')
routes = ''.join(findroutes.findall(raw_html_routes.text))
routes = getroutes.findall(routes)
raw_html_stops_in_routes = {i: requests.get('https://www.bustime.ru/vladivostok/bus-'+i+'/') for i in routes}
cnx = mysql.connector.connect(user = "MNIXenus", password = "00mefomu", host = "127.0.0.1", database = "buses")
cursor = cnx.cursor()
findstops_routes = re.compile('<table class="ui compact unstackable table"(.+?)</table>', re.S)
getstops_routes = re.compile('<td>(.+?)</td>', re.S)
stops_in_routes = {}
for i in raw_html_stops_in_routes:
    stops_in_route = findstops_routes.findall(raw_html_stops_in_routes[i].text)[0]
    stops_in_route = getstops_routes.findall(stops_in_route)
    stops_in_route = [BeautifulSoup(i, "html5lib").get_text().rstrip() for i in stops_in_route]
    stops_in_route = [i for i in stops_in_route if i.rstrip('0123456789:') != '']
    stops_in_route[0] = stops_in_route[0].rstrip('0123456789:')
    for j in range(len(stops_in_route)):
        cursor.execute('SELECT idstops FROM stops WHERE name= "'+stops_in_route[j]+'"')
        result = cursor.fetchall()
        if result != []: 
            stops_in_route[j] = result[0][0]
        else:
            stops_in_route[j] = 0   
    stops_in_routes[i] = stops_in_route
cnx.commit()
cursor.close()
cnx.close()

In [31]:
#заполнить маршруты
cnx = mysql.connector.connect(user = "MNIXenus", password = "00mefomu", host = "127.0.0.1", database = "buses")
cursor = cnx.cursor()
for i in range(len(routes)):
    route = (routes[i], i)
    cursor.execute(add_route.format(*route))
for i in range(max([len(stops_in_routes[i]) for i in stops_in_routes])):
    cursor.execute('ALTER TABLE routes ADD stop_'+str(i)+' INT;')
cnx.commit()
for i in range(max([len(stops_in_routes[i]) for i in stops_in_routes])):
    cursor.execute('ALTER TABLE routes ADD FOREIGN KEY (stop_'+str(i)+') REFERENCES stops(idstops);')
cnx.commit()
for i in stops_in_routes:
    counter = 0
    for j in stops_in_routes[i]:
        cursor.execute('UPDATE routes SET stop_'+str(counter)+'="'+str(j)+'" WHERE idroutes = "'+str(i)+'";')
        counter +=1
cnx.commit()
cursor.close()
cnx.close()

In [32]:
#заполнить дистанции
cnx = mysql.connector.connect(user = "MNIXenus", password = "00mefomu", host = "127.0.0.1", database = "buses")
cursor = cnx.cursor()
num_stops = range(len(stops))
for i in num_stops:
    cursor.execute('ALTER TABLE distances ADD stop_'+str(i)+' INT;')

distances_matrix = [[0 for i in num_stops] for j in num_stops]
for i in num_stops:
    for j in num_stops:
        distances_matrix[i][j] = distances_matrix[j][i] = random.randint(100, 5000)
for i in num_stops:
    cursor.execute('INSERT INTO distances VALUES('+str(i)+', '+str(distances_matrix[i])[1:-1]+');')
cnx.commit()
cursor.close()
cnx.close()

In [33]:
#заполнить показания датчиков
cnx = mysql.connector.connect(user = "MNIXenus", password = "00mefomu", host = "127.0.0.1", database = "buses")
cursor = cnx.cursor()
cursor.execute('SELECT idroutes FROM routes;')
routes = [i[0] for i in cursor.fetchall()]
cursor.execute('SELECT * FROM routes;')
result = cursor.fetchall()
stops_in_routes = {i[0]:list(i[1:]) for i in result}
for i in stops_in_routes: #clean away from None
    stops_in_routes[i] = [i for i in stops_in_routes[i] if i != None]
sensors_data = [[i, j, stops_in_routes[i][1]] for i in routes for j in stops_in_routes[i][2:]]
counter = 0
for i in routes:
    arrival_time = datetime.datetime(2019, 5, 20, 10, 10, 50)
    for j in stops_in_routes[i][2:]:
        departure_time = arrival_time + datetime.timedelta(seconds = random.randint(0, 300))
        sensors_data[counter].append(str(arrival_time))
        sensors_data[counter].append(str(departure_time))
        arrival_time = departure_time + datetime.timedelta(seconds = random.randint(100, 400))
        counter += 1
for i in range(len(sensors_data)):
    cursor.execute(add_sensor_data.format(i, sensors_data[i][0], sensors_data[i][1],  str(sensors_data[i][2]), sensors_data[i][3], sensors_data[i][4]))
cnx.commit()
cursor.close()
cnx.close()

In [22]:
print(*sensors_data[1])

35 255 0 2019-05-20 10:18:07 2019-05-20 10:18:22
